In [17]:
import pandas as pd
import numpy as pd

import pickle
import joblib 

from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import read
from scipy.fftpack import fft
import pydub
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import speech_recognition as sr

In [29]:
data2 = pd.read_csv("./Medley-solos-DB/Medley-solos-DB_metadata.csv")
df2 = pd.DataFrame(data2)
df2.shape

(21571, 5)

In [32]:
validation = df2["subset"]=="validation"
df_validation= df2[validation]
df_validation.shape

(3494, 5)

In [33]:
#df_validation = df_validation.apply(lambda df: f"./Medley-solos-DB_validation-{df['instrument_id']}{df['uuid4']}.wav")

df_validation['uuid4'] = df_validation.apply(lambda row: f"./Medley-solos-DB/Medley-solos-DB_validation-{row.instrument_id}_{row.uuid4}.wav", axis = 1)

df_validation.head()

<ipython-input-33-4780b8b1f3cc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_validation['uuid4'] = df_validation.apply(lambda row: f"./Medley-solos-DB/Medley-solos-DB_validation-{row.instrument_id}_{row.uuid4}.wav", axis = 1)


,subset,instrument,instrument_id,song_id,uuid4
18077,validation,clarinet,0,200,./Medley-solos-DB/Medley-solos-DB_validation-0...
18078,validation,clarinet,0,200,./Medley-solos-DB/Medley-solos-DB_validation-0...
18079,validation,clarinet,0,200,./Medley-solos-DB/Medley-solos-DB_validation-0...
18080,validation,clarinet,0,200,./Medley-solos-DB/Medley-solos-DB_validation-0...
18081,validation,clarinet,0,200,./Medley-solos-DB/Medley-solos-DB_validation-0...


In [41]:
instrument_lst = set(df_validation.instrument.values)  
instrument_lst

{'clarinet',
 'distorted electric guitar',
 'female singer',
 'flute',
 'piano',
 'tenor saxophone',
 'trumpet',
 'violin'}

In [42]:
instrument_lst = list(instrument_lst)

In [43]:
instruments_validation_paths = {}
for inst in instrument_lst:
    instruments_validation_paths.update({inst:list(df_validation['uuid4'][df_validation["instrument"]==inst])})



In [46]:
# Creamos una copia de instruments_test_paths por si modificamos algo por error,
# para no tener que volver a ejecutar la celda anterior.
instruments_validation_paths_copy = instruments_validation_paths

In [47]:
def removeEmptyPaths(inst_paths_dict):
    '''
    Esta función limpia los instrumentos cuya lista
    de paths esté vacía y te devuelve una copia del diccionario limpio.
    '''
    inst_paths_dict_cleaned = inst_paths_dict
    for inst, paths in inst_paths_dict.items():
        if not paths:
            inst_paths_dict_cleaned.pop(inst)
        else:
            print(inst)
    return inst_paths_dict_cleaned

In [48]:
instruments_validation_paths_copy = removeEmptyPaths(instruments_validation_paths_copy)

distorted electric guitar
tenor saxophone
trumpet
piano
flute
clarinet
female singer
violin


In [63]:
def normalizeInstPathsLen(inst_paths_dict):
    '''
    Esta función va a homogeneizar la longitud 
    de las listas de paths que tiene el diccionario
    que recibe, va a crear un nuevo diccionario y 
    lo va a devolver.
    '''
    insts_paths_normalized_len = {}
    min_len = len(list(inst_paths_dict.items())[0][1])
    for inst, paths in inst_paths_dict.items():
        if min_len > len(paths):
            min_len = len(paths)
    for inst, paths in inst_paths_dict.items():
        insts_paths_normalized_len[inst] = paths[:min_len]
        
    return insts_paths_normalized_len

In [64]:
insts_validation_paths_normalized = normalizeInstPathsLen(instruments_validation_paths_copy)

In [65]:
insts_validation_paths_normalized

{'distorted electric guitar': ['./Medley-solos-DB/Medley-solos-DB_validation-1_dfbe2293-1071-59e0-ff2b-5a506185645b.wav',
  './Medley-solos-DB/Medley-solos-DB_validation-1_4b11a51c-05ed-581c-f4da-e62dc0b5c9aa.wav',
  './Medley-solos-DB/Medley-solos-DB_validation-1_385bb265-2880-5f32-f792-e48aab6ff982.wav',
  './Medley-solos-DB/Medley-solos-DB_validation-1_ed9d1923-8824-52f8-ff3d-705d8a615dd7.wav',
  './Medley-solos-DB/Medley-solos-DB_validation-1_3bc0d040-a61b-5d85-f00d-339f0dea9946.wav',
  './Medley-solos-DB/Medley-solos-DB_validation-1_28162e9f-78f9-5e13-fdf6-afe91fb71534.wav',
  './Medley-solos-DB/Medley-solos-DB_validation-1_a0c452d5-2275-5d2f-f2c8-606b7b1119c7.wav',
  './Medley-solos-DB/Medley-solos-DB_validation-1_eb822319-cedd-5f7a-ff02-9334495f8c7a.wav',
  './Medley-solos-DB/Medley-solos-DB_validation-1_7211f457-4ec3-5299-fa6a-a64ceeac20c0.wav',
  './Medley-solos-DB/Medley-solos-DB_validation-1_93eeb0af-e62c-50d8-fc8e-37951a116d04.wav',
  './Medley-solos-DB/Medley-solos-DB_vali

In [70]:
def samplesList(path_lst):
    '''
    Esta función crea una lista de objetos de AudioSegment
    por cada fichero de la lista que recibe (path_lst),
    después por cada objeto AudioSegment, obtiene la lista
    de samples de cada audio, creando una nueva lista y devolviéndola.
    '''
    audio_lst = []
    audio_samples_lst = []
    for path in path_lst:
        audio = AudioSegment.from_file(path, format='wav')
        audio_lst.append(audio)
    for a in audio_lst:
        audio_samples_lst.append(a.get_array_of_samples())
    
    return audio_samples_lst

def fftList(samples_lst):
    '''
    Esta función recibe una lista de samples de audios,
    hace la transformada de Fourier y devuelve una lista, 
    con toda la lista transformada.
    '''
    fft_lst = []
    for sample in samples_lst:
        four = abs(fft(sample))
        fft_lst.append(four)
    
    return fft_lst

In [71]:
# Cogemos el diccionario y por cada instrumento generamos su lista de samples:

def instSamples(insts_paths):
    '''
    Esta función genera un dict de instrumentos
    y su lista samples a partir del dict de paths 
    que recibe y lo devuelve.
    '''
    inst_samples = {}
    for inst, paths in insts_paths.items():
        inst_samples[inst] = samplesList(paths)
        
    return inst_samples

In [72]:
insts_validation_samples = instSamples(insts_validation_paths_normalized)

In [73]:
def infoSamples(inst_samples):
    '''
    Esta función halla la longitud máxima, mínima y
    la media de los samples de todos los instrumentos.
    '''
    min_len = len(list(inst_samples.items())[0][1][0])
    max_len = 0
    joined_samples_lst = []
    for inst, samples_lst in inst_samples.items():
        joined_samples_lst = joined_samples_lst + samples_lst
        
    means_lst = []
    for samples in joined_samples_lst:
        samples_len = len(samples)
        means_lst.append(samples_len)
        if min_len > samples_len:
            min_len = samples_len
        if max_len < samples_len:
            max_len = samples_len
    
    print(f'Mean: {sum(means_lst) / len(means_lst)}')
    print(f'Max len: {max_len}')
    print(f'Min len: {min_len}')

In [74]:
infoSamples(insts_validation_samples)

Mean: 131072.0
Max len: 131072
Min len: 131072


In [75]:
def instFft(insts_samples):
    '''
    Esta función genera un dict de instrumentos
    y su lista samples a partir del dict de paths 
    que recibe y lo devuelve.
    '''
    inst_ffts = {}
    for inst, samples in insts_samples.items():
        inst_ffts[inst] = fftList(samples)
        
    return inst_ffts

insts_ffts = instFft(insts_validation_samples)

In [76]:
def dataFramer(insts_ffts):
    '''
    Esta función toma el diccionario que se le pase como argumento
    y, por cada entrada del diccionario, crea un dataframe y los concatena 
    todos.
    '''
    lst_df = []
    for inst, fft_lst in insts_ffts.items():
        inst_val = {"transformed": fft_lst, "type": inst}
        lst_df.append(pd.DataFrame(inst_val))
    return pd.concat(lst_df).reset_index(drop=True)

In [77]:
df_total_validation = dataFramer(insts_ffts)

In [78]:
df_total_validation.head()

,transformed,type
0,"[1810399232.0, 2121176134.2477577, 2014916240....",distorted electric guitar
1,"[1645543424.0, 1315648156.6486316, 1311040667....",distorted electric guitar
2,"[2435219456.0, 2057058782.5004284, 2101994106....",distorted electric guitar
3,"[3655565312.0, 3342861768.674649, 3394058895.9...",distorted electric guitar
4,"[1179746304.0, 875337939.5570114, 865450750.90...",distorted electric guitar


In [79]:
df_total_validation.to_pickle(r'./validation-bueno.csv')